In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import folium
from folium import plugins
from folium.plugins import MarkerCluster
from geopy.distance import geodesic
from geopy.geocoders import Nominatim


In [3]:
bike_master = pd.read_csv('bike_full.csv', dtype={5: str, 7: str})  # Convert both columns to strings

In [4]:
# before_cleaning = bike_master.memory_usage(deep=True).sum() / 1024 ** 2
# print(f"Memory usage before cleaning: {before_cleaning:.2f} MB")

In [5]:
bike_master.isnull().sum()

ride_id                    0
rideable_type              0
started_at                 0
ended_at                   0
start_station_name    742776
start_station_id      742776
end_station_name      811282
end_station_id        811282
start_lat                  2
start_lng                  2
end_lat                16765
end_lng                16765
member_casual              0
dtype: int64

In [6]:
bike_master_clean = bike_master.copy()

# Convert datetime columns
bike_master_clean['started_at'] = pd.to_datetime(bike_master_clean['started_at'])
bike_master_clean['ended_at'] = pd.to_datetime(bike_master_clean['ended_at'])

# Convert category columns
category_columns = ['rideable_type', 'start_station_name', 'end_station_name', 'member_casual']
for col in category_columns:
    bike_master_clean[col] = bike_master_clean[col].astype('category')

# Convert string columns (IDs)
string_columns = ['ride_id', 'start_station_id', 'end_station_id']
for col in string_columns:
    bike_master_clean[col] = bike_master_clean[col].astype('string')



In [7]:
after_cleaning = bike_master_clean.memory_usage(deep=True).sum() / 1024 ** 2
print(f"Memory usage after cleaning: {after_cleaning:.2f} MB")

Memory usage after cleaning: 2355.93 MB


In [8]:
bike_master_clean.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,5CB9DFCECF79AF84,classic_bike,2021-01-01 00:08:33,2021-01-01 00:33:53,Maine Ave & 9th St SW,31646.0,Rosslyn Metro / Wilson Blvd & Ft Myer Dr,31015.0,38.880440,-77.025236,38.894600,-77.072305,member
1,629E059504606547,electric_bike,2021-01-01 00:13:43,2021-01-01 00:29:34,10th & U St NW,31111.0,NaN,<NA>,38.917193,-77.025894,38.960000,-77.020000,casual
2,E74069873161EE33,electric_bike,2021-01-01 00:14:32,2021-01-01 00:28:45,17th & Corcoran St NW,31214.0,14th & Belmont St NW,31119.0,38.912138,-77.038568,38.920870,-77.031691,member
3,91F95E512CABC46A,classic_bike,2021-01-01 00:15:45,2021-01-01 00:21:20,Wilson Blvd. & N. Vermont St.,31926.0,Wilson Blvd. & N. Vermont St.,31926.0,38.879477,-77.114563,38.879477,-77.114563,member
4,DA46A05139C0EA2F,classic_bike,2021-01-01 00:17:46,2021-01-01 00:21:00,11th & Park Rd NW,31651.0,14th & Newton St NW,31649.0,38.931322,-77.028247,38.931991,-77.032956,member


In [9]:
bike_master_clean.isnull().sum()

ride_id                    0
rideable_type              0
started_at                 0
ended_at                   0
start_station_name    742776
start_station_id      742776
end_station_name      811282
end_station_id        811282
start_lat                  2
start_lng                  2
end_lat                16765
end_lng                16765
member_casual              0
dtype: int64

In [10]:
bike_master_clean[['start_station_name', 'end_station_name', 'start_station_id', 'end_station_id','start_lat']].nunique()

start_station_name       860
end_station_name         865
start_station_id        1321
end_station_id          1338
start_lat             570110
dtype: int64

In [11]:
# # Find and export matching stations
# matching_station = bike_master_clean[
#   (bike_master_clean['start_lat'].round(2) == 38.93) & 
#   (bike_master_clean['start_lng'].round(2) == -77.02)
# ][['start_station_name', 'start_lat', 'start_lng']].dropna().drop_duplicates()

# matching_station.to_csv('matching_stations1.csv', index=False)

In [12]:
bike_master_clean.isnull().sum()

ride_id                    0
rideable_type              0
started_at                 0
ended_at                   0
start_station_name    742776
start_station_id      742776
end_station_name      811282
end_station_id        811282
start_lat                  2
start_lng                  2
end_lat                16765
end_lng                16765
member_casual              0
dtype: int64

In [ ]:
#create a new column for month year
bike_master_clean['month_year'] = bike_master_clean['started_at'].dt.to_period('M')

#create a new column for year
bike_master_clean['year'] = bike_master_clean['started_at'].dt.year

#create a new dataframe for the month june 2023 to december 2023
bike_master_clean_2023 = bike_master_clean[(bike_master_clean['year'] == 2023) & (bike_master_clean['month_year'] >= '2023-10')]

row_count = len(bike_master_clean_2023)
print(f"Number of rows in the dataset: {row_count}")



Number of rows in the dataset: 1126727


In [34]:
#find missing null values
bike_master_clean_2023.isnull().sum()

#drop missing values
bike_master_clean_2023 = bike_master_clean_2023.dropna()


In [41]:
#get the top 10 stations with the highest number of trips using the start station name and ride id
top_stations = (
    bike_master_clean_2023
    .groupby('start_station_name',observed=False)
    .agg(trip_count=('ride_id', 'count'))
    .nlargest(10, 'trip_count')
    .reset_index()
)

top_stations

,start_station_name,trip_count
0,Columbus Circle / Union Station,11813
1,New Hampshire Ave & T St NW,9614
2,15th & P St NW,8504
3,Eastern Market Metro / Pennsylvania Ave & 8th ...,8108
4,5th & K St NW,7944
5,1st & M St NE,7636
6,14th & V St NW,7559
7,M St & Delaware Ave NE,7402
8,Massachusetts Ave & Dupont Circle NW,7316
9,8th & O St NW,6407


In [60]:
# Filter for the specific start station name
station_name = 'Columbus Circle / Union Station'
df_filtered = bike_master_clean_2023[bike_master_clean_2023['start_station_name'] == station_name]

#count the number of entries for the specific station
df_filtered_count = df_filtered.shape[0]

# Count unique start latitude and longitude values
num_unique_lat_lng = df_filtered[['start_lat', 'start_lng']].drop_duplicates().shape[0]

#Unique start latitude and longitude values
unique_lat_lng = df_filtered[['start_lat', 'start_lng']].drop_duplicates()
unique_lat_lng



,start_lat,start_lng
9567489,38.897,-77.005
9812020,38.887,-77.002
10009125,38.897,-77.004
10052987,38.964,-77.010
10109678,38.898,-77.005
10411046,38.955,-77.009
10426656,38.896,-77.005
10478355,38.888,-77.008
10518933,38.939,-77.037
10580481,38.881,-76.993


In [67]:
# Calculate the mean latitude and longitude for each start station
mean_coords = bike_master_clean_2023.groupby('start_station_name', observed=False)[['start_lat', 'start_lng']].mean().reset_index()

# Sort by the number of occurrences to find the top 10 most frequent start locations
top_10 = bike_master_clean_2023.groupby('start_station_name',observed=False).size().reset_index(name='count')
top_10 = top_10.sort_values(by='count', ascending=False).head(10)

# Merge the mean coordinates with the top 10 locations
top_10_coords = top_10.merge(mean_coords, on='start_station_name')
top_10_coords

# Create a base map centered around the mean coordinates of the top 10 locations
map_center = [top_10_coords['start_lat'].mean(), top_10_coords['start_lng'].mean()]
map = folium.Map(location=map_center, zoom_start=13)

# Add a marker for each of the top 10 locations
for i in range(top_10_coords.shape[0]):
    row = top_10_coords.iloc[i]
    folium.Marker(
        location=[row['start_lat'], row['start_lng']],
        popup=row['start_station_name'],
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(map)   
map
